# Introduction

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from Bio import SeqIO

# On définit les séquences qu'on veut récupérer
# On les nomme en indice pour savoir quelle est la première, la seconde, etc...
SEQUENCES = {
  'HOMME': 'SARS-Cov2 [ORGN] AND srcdb_refseq [PROP]',
  'CHAUVE-SOURIS': 'SARSr-CoV RaTG13',
  'PANGOLIN': 'Pangolin coronavirus isolate MP789 MT121216',
}

## Partie A

In [ ]:
Entrez.email = 'mathieu@colmon.fr'

# On fait une fonction pour récupérer le premier
# résultat d'une requête sous forme d'un objet Seq
def getSeq(request):
  # On exécute la requête demandée
  searchReq = Entrez.esearch(db='nucleotide', term=request)
  # On lit le résultat
  searchRes = Entrez.read(searchReq)
  # On ferme la requête
  searchReq.close()

  # On demande le premier résultat
  firstReq = Entrez.efetch(db='nucleotide', id=searchRes['IdList'], rettype='gb')
  # On lit le résultat
  firstRes = SeqIO.read(firstReq, 'gb')
  # On ferme la requête
  firstReq.close()

  return firstRes

# On crée une liste qui stockera les séquences
SEQs = []

# Pour chaque séquence demandée
for sName in SEQUENCES:
  # On traite la requête et on stocke le résustat dans 'SEQs'
  SEQs.append(getSeq(SEQUENCES[sName]))

# On écrit le résultat dans le fichier 'seq_covid.gb'
SeqIO.write(SEQs, './sequences/seq_covid.gb', 'genbank')

# Log #
print('Séquences des coronavirus écrites dans le fichier \'./sequences/seq_covid.gb\' !')

## Partie B

In [ ]:
SEQs = SeqIO.parse('./sequences/seq_covid.gb', 'genbank')

SPIKEs = []

# Pour chaque séquence de coronavirus
for seq in SEQs:
  for f in seq.features:
    # Si le type est 'gene', on vérifie si son nom est 'S'
    if f.type == 'gene' and f.qualifiers.get('gene')[0] == 'S':
      # On ajoute à la liste de séquences 'SPIKEs'
      SPIKEs.append(f.translate(seq))

# On écrit le contenu de 'SPIKEs' dans le fichier 'spike.fasta'
SeqIO.write(SPIKEs, './sequences/spike.fasta', 'fasta')

# Log #
print('Séquences des gènes écrites dans le fichier \'./sequences/spike.fasta\' !')

## Partie C

In [ ]:
# On importe le module permettant d'aligner les séquences
from Bio.Align.Applications import MafftCommandline
from os import path
from platform import system

In [ ]:
# Par défaut (sur Linux) le moyen d'accéder à MAFFT est la commande 'mafft'
mafftPath = 'mafft'

# Sur Windows, MAFFT n'est accessible depuis la commande 'mafft'.
if system()== 'Windows':
  # On cherche donc le chemin d'accès absolu du fichier './mafft-win/mafft.bat'
  mafftPath = path.abspath('./mafft-win/mafft.bat')

# On Aligne les séquences du fichier 'spike.fasta'
stdout, stderr = MafftCommandline(cmd=mafftPath, input='./sequences/spike.fasta')()

# On écrit le résultat dans 'aln-spike.fasta'
with open('./sequences/aln-spike.fasta', 'w') as w:
  w.write(stdout)

# Log #
print('Séquences alignées dans le fichier \'./sequences/aln-spike.fasta\' !')

## Partie D

In [ ]:
# On récupère toutes les séquences alignées
SEQs = SeqIO.parse('./sequences/aln-spike.fasta', 'fasta')

# On crée une liste qui contiendra les séquences sous forme de chaines de caractères
rawSeqs = []

# Pour chaque séquence alignée
for seq in SEQs:
  # On stocke la séquence sous forme de chaine de caractère dans 'rawSeqs'
  rawSeqs.append(seq.seq)

# On crée une liste qui contiendra les noms des colonnes du tableau résultat
indexes = list(SEQUENCES.keys())
# La première colonne est celle de la position
indexes.insert(0, 'POSITION')

# On ouvre le fichier qui contiendra les résultats des comparaisons
file = open('./sequences/resultatComparaison_geneS.txt', 'w')

# On stocke le nombre de lignes écrites dans une variable de manière à
# pouvoir afficher toutes les n (50) lignes les noms des indices
nbLines = 0

# On crée une fonction qui permet à la fois d'afficher une ligne
# dans la console, et de stocker la ligne dans le fichier résultat
def printTableLine(data):
  # On importe la variable globale 'nbLines'
  global nbLines
  # On incrémente la variable 'nbLines'
  nbLines += 1
  # Si l'indice de la ligne (nbLines - 1) est un multiple de 50
  if (nbLines - 1) % 50 == 0:
    # On affiche et stocke les indices
    printTableLine(indexes)

  # On formatte la ligne du tableau
  formattedRow = '{: >8} {: >15} {: >15} {: >15}'.format(*data)
  # On l'écrit dans le fichier résultat
  file.write(formattedRow + '\n')
  # On l'écrit dans la console
  print(formattedRow)

# Les séquences sont alignées donc elles ont le même nombre de caractères.
# Pour chaque caractère de la (première) séquence
for pos in range(len(rawSeqs[0]) - 1):
  # Pour chaque séquence...
  for rawSeq in rawSeqs:
    # S'il y a une différence du n ième caractère entre
    # la première séquence et la séquence 'rawSeq'
    if rawSeq[pos] != rawSeqs[0][pos]:
      # On crée une liste qui correspond au changement du n ième caractère
      change = [pos]
      # Pour chaque indice de la liste des séquences alignées
      for iSeq in range(len(rawSeqs)):
        # On ajoute les changements dans la liste
        change.append(rawSeqs[iSeq][pos])

      # On ajoute la ligne au tableau des changements
      printTableLine(change)
      # On passe au caractère suivant
      break

# On ferme le fichier
file.close()